This notebook shows some example queries ontop of the example dataset. First of all we load all the data into the sql context:

In [27]:
for (set <- Seq("business", "checkin", "photos", "review", "user", "tip")) {
    spark.read.parquet(s"/data/parquet/$set").createOrReplaceTempView(set)
}

lastException: Throwable = null


---
For example, as users have a review count, we can look if that count matches the count of reviews by users inside the review data set:

In [93]:
val diff = spark.sql("""
SELECT user.review_count - review.review_count AS diff, count(1) AS diff_num_occured 
FROM user JOIN 
     (SELECT user_id, count(1) as review_count 
      FROM review 
      GROUP BY user_id) AS review
ON user.user_id == review.user_id
GROUP BY diff
""")

diff = [diff: bigint, diff_num_occured: bigint]


[diff: bigint, diff_num_occured: bigint]

In [94]:
diff.head(6)

29,4534
26,5302
474,22
964,5
2250,1
2214,1


In [95]:
diff.filter("diff<=0").head(6)

0,390381
-3,108
-35,1
-60,1
-25,3
-49,3


This clearly shows that there are many differecies, mostly missing reviews in the review dataset, but for some users, the count of reviews inside the review dataset is higher as the review_count attriute of the user. 

---

We can also look if all bussniss_id's inside checkin, photos, reviews and tip are matchin the business dataset:

In [91]:
def missing(set: String): String = s"""
SELECT sum(IF(business.business_id IS NULL, 1, 0)) as missing_business,
       sum(IF($set.business_id IS NULL, 1, 0)) as missing_$set
FROM business FULL OUTER JOIN $set
ON business.business_id == $set.business_id
"""

spark.sql(missing("review"))

missing: (set: String)String


[missing_business: bigint, missing_review: bigint]

In [92]:
spark.sql(missing("checkin")).head(1)

0,28217


In [88]:
spark.sql(missing("photos")).head(1)

0,145470


In [89]:
spark.sql(missing("review")).head(1)

0,0


In [90]:
spark.sql(missing("tip")).head(1)

0,62201


So here we can see that no entry form the other sets is missing inside the bussiness set. Furthermore all businesses have at least one review.